# Семнар 2

В данном семинаре будем классифицировать твиты на те, которые нравятся/не нравятся людям

# Данные


In [1]:
import numpy as np

Download dependings

In [3]:
import pip

pip.main(['install', 'tqdm'])

0

In [4]:
from tqdm import tqdm

In [5]:
import pandas
tweets = pandas.DataFrame.from_csv("Tweets data.csv", encoding = "ISO-8859-1")

In [6]:
print(tweets[::5000])

        Sentiment                                      SentimentText
ItemID                                                              
1               0                       is so sad for my APL frie...
5002            1       very very very happy and slightly skinny  xx
10012           1  &quot;I feel better now, better than James Bro...
15012           1  ...what can I say ....what a surprise...http:/...
20012           0                                     @ ponyp: hey! 
25012           1       @404world yes I'm going to find one..pronto 
30012           0  @AdamJonesey mmm id love some parma violets ri...
35012           1  @AlTheYid Roast beef and wasabi... yummmmmmmmm...
40012           0  @amandafortier ahhh. yeah. kinda. I tweet from...
45012           0                 @antdeshawn ... the pain is worst 
50012           1  @Ashtarte Just be your usual supportive self. ...
55012           0  @azizabatini86 and that's fine--u kno, the dry...
60012           0       @Bellemord

# Preprocessing


In [8]:
import re

In [9]:
texts = tweets["SentimentText"]
Y = tweets["Sentiment"]

In [10]:
texts.index = np.arange(texts.shape[0])
Y.index = np.arange(Y.shape[0])

Посмотрим на твит:

In [11]:
texts[3]

"          .. Omgaga. Im sooo  im gunna CRy. I've been at this dentist since 11.. I was suposed 2 just get a crown put on (30mins)..."

Давайте приведём текст твита в нижний регистр, а также избавимся от всех символов, кроме английских букв:



In [12]:
re.sub(r'[^a-zA-Z ]+', '', texts[3].lower())

'           omgaga im sooo  im gunna cry ive been at this dentist since  i was suposed  just get a crown put on mins'

In [14]:
texts = list(map(lambda r: re.sub(r'[^a-zA-Z ]+', '', r.lower()), texts))

In [15]:
texts = np.array(texts)

In [16]:
texts[:4]

array(['                     is so sad for my apl friend',
       '                   i missed the new moon trailer',
       '              omg its already  o',
       '           omgaga im sooo  im gunna cry ive been at this dentist since  i was suposed  just get a crown put on mins'], 
      dtype='<U755')

## Model 0. Признаки - буквы.

Возьмем в качестве признаков буквы и опишем каждый твит, как количевство вхождений каждой буквы в него

In [17]:
def extract_features(text):
    features = []
    letters = "abcdefghijklmnopqrstuvwxyz"    
    for l in letters:
        features.append(text.count(l))  
    return features

In [18]:
import numpy
X = []
for text in texts:
    X.append(extract_features(text))
X = numpy.array(X)



Научим на наших признаках простую линейную модель

In [25]:
train_size = int(len(X) * 0.7)

In [26]:
#порежем данные
X_train = X[:train_size]
Y_train = Y[:train_size]
X_test = X[train_size:]
Y_test = Y[train_size:]

In [27]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train, Y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [28]:
from sklearn.metrics import roc_auc_score

In [29]:
Y_pred_train = lr.predict_proba(X_train)[:, 1]
Y_pred_test = lr.predict_proba(X_test)[:, 1]

print("Обучение:",roc_auc_score(Y_train,(Y_pred_train > 0.5)))
print("Тест:", roc_auc_score(Y_test,(Y_pred_test > 0.5)))


Обучение: 0.553202961799
Тест: 0.552706238528


#  Opinion mining

Давайте использовуем нашу модель чтобы понять, что людям сейчас нравится/не нравится

In [30]:
import numpy
with open("testdata.txt") as fin:
    test_tweets = fin.read()[:-1].split('\n')
test_tweets = numpy.array(test_tweets)

In [31]:
X_ops = []
for tweet in test_tweets:
    X_ops.append(extract_features(tweet))
X_ops = numpy.array(X_ops)


In [32]:
#sentiments = <предскажите эмоциональную окраску x_ops>
sentiments = lr.predict_proba(X_ops)[:,1]
sentiment_order = numpy.argsort(-sentiments)

Теперь посмотрим на наиболее любимые/ненавистные людям вещи

In [34]:
best_ids = sentiment_order[:10]

for tweet, score in zip(test_tweets[best_ids], sentiments[best_ids]):
    print(score,':',tweet, '\n')

0.901476836514 : i love sa manthaaaaa aaa aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa aaaaaaaaaaaaaaaa.. < nononeofyouknowherandit'snottheoneyourthinkingabout >.. 

0.882174650388 : gnun ata talaga pag nageenejoy ka at ngeenjoy ka sa mga kasama mo. you guys make every difficult things in school easy, kc alam ko nandyan kyo to back everyone up. 

0.826829838596 : I really like the Davis Square, Porter Square, and Harvard Square areas. 

0.815749205072 : apparently Harvard College is the undergraduate program of Harvard University, i thought Harvard College was some crappy college playing off of the real thing.. 

0.805262833876 : * ugly * Mit s5 auf DVD von Angel hab ich ja jetzt endlich alle Buffy UND Angel Staffeln auf DVd, yay!: 

0.797770466925 : u baggin on my boy jake cuz if u is i could straight fuck you azz up lil bitch holla ata playa.. 

0.79627790423 : AAA ya se empezare dandoles la grata noticia q se me fue la caspa q tenia detras de la cabeza aquel horrible dia. 

0.785457638773 : You'

In [36]:
worst_ids = sentiment_order[-10:]

for tweet, score in zip(test_tweets[worst_ids], sentiments[worst_ids]):
    print(score,':',tweet, '\n')

0.268062821465 : but NO. no one posted'cept cory saying somtine was up my butt how rude i just mit shut this site down so i don't have to deal with it... 

0.266235597401 : stupid stupid stupid stupid stupid stupid southwest airlines. 

0.261735544188 : I spent a good bit of time racing around the city on my bike, and last night I went to see Mission Impossible 3 since Tom Cruise is awesome. 

0.249126714331 : " You know every time I come around your London, London Bridge, it's goin down like London, London, London, it's goin down I'm comin, comin, comin... 

0.243382192239 : substantiated debt consolidation loans gm mastercard customizing debt elimination He's simply too inept otherwise.. 

0.237165226724 : I wanted to test drive both the Stang and the Scion, but when we hit the Toyota lot I spotted the stalker man from my previous visit and busted an immediate bitch. 

0.231410601486 : the first time, i got to spend time with sansan and tito robbie in the san francisco museum of mode

## Model 1. Bag-of-words & Logistic Regression

Аналогично предыдущей модели будем описывать текст, как вектор из количества вхождений каждого признака, только теперь вместо букв, признаками будут являться слова.

In [37]:
from sklearn.feature_extraction.text import CountVectorizer

In [39]:
vectorizer = CountVectorizer(min_df=5)
_ = vectorizer.fit(texts)

In [40]:
X = vectorizer.transform(texts)

In [41]:
from sklearn.cross_validation import ShuffleSplit
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, accuracy_score

/home/tvorogme/.anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Будем тестировать модель с помощью кросс-валидации:

In [42]:
cv = ShuffleSplit(X.shape[0], n_iter=3, test_size=0.3)
for train_ids, test_ids in cv:
    lr = LogisticRegression()
    lr.fit(X[train_ids], Y[train_ids])
    preds = lr.predict_proba(X[test_ids])[:,1]
    print('ROC-AUC: %.3f, ACC: %.3f' % (roc_auc_score(Y[test_ids], preds), 
                                        accuracy_score(Y[test_ids], (preds > 0.5).astype(int))))

ROC-AUC: 0.836, ACC: 0.763
ROC-AUC: 0.838, ACC: 0.764
ROC-AUC: 0.840, ACC: 0.767


## Важность признаков

Как уже упоминалось ранее, веса признаков в линейной модели в случае, если признаки отмасштабированы, характеризуют степень их влияния на значение целевой переменной. В задаче классификации текстов, кроме того, признаки являются хорошо интерпретируемыми, поскольку каждый из них соответствует конкретному слову. Изучим влияние конкретных слов на значение целевой переменной:

In [47]:
f_weights = zip(vectorizer.get_feature_names(), lr.coef_[0])
f_weights = sorted(f_weights, key=lambda i: i[1])

In [48]:
for i in range(1,30):
    print("{0}, {1}".format(f_weights[-i][0], f_weights[-i][1]))

musicmonday, 2.560364951179168
worries, 2.167659833533782
congratulations, 2.0753534207506212
chuckle, 2.0032833341327168
woohoo, 1.9899849963193705
critic, 1.9843612781839177
wwwiamsoannoyedcom, 1.9592470065670284
followfriday, 1.9269647960484764
welcome, 1.925830347196484
doll, 1.8618247680085374
boyislost, 1.8513132707008306
quotit, 1.8497473342272837
goodsex, 1.8460538038797283
yey, 1.8179764729054693
smiling, 1.781149065263116
bellacullen, 1.7757361013107607
cloudstrife, 1.7658511676341042
annjj, 1.733208788645981
booshtukka, 1.7319636823489437
stoked, 1.7247584291060731
satisfied, 1.6806216524716966
thank, 1.6802451474639342
combustiblesong, 1.6744409245966165
arabidopsis, 1.6476211492744646
launch, 1.6474884814853406
aileenu, 1.6396597498964292
cities, 1.6171500868242967
bing, 1.6134311955138243
heh, 1.6035984178309213


In [49]:
for i in reversed(range(1,30)):
    print("{0}, {1}".format(f_weights[i][0], f_weights[i][1]))

hurting, -1.9281762034130823
fml, -1.9357244124150907
unfortunate, -1.9401259010058471
boooo, -1.94208181758656
broken, -1.9444756326085972
hurts, -1.956152551553251
pakcricket, -2.0084920441404255
sick, -2.0157574339105184
heartbreaking, -2.070465744505251
pricey, -2.0776284668767446
ruined, -2.09427070007355
storms, -2.1003701909114287
amazingphoebe, -2.105247967257527
unfortunately, -2.1133462280121877
argh, -2.1204122192984802
miss, -2.137825575143884
nooo, -2.2365164777141993
cancelled, -2.255755521647335
frustrating, -2.263412179194482
missing, -2.2865498498114634
depressing, -2.2891605529180796
rip, -2.2982709259545375
sadly, -2.4360409622155084
gutted, -2.4364451664095244
poor, -2.4703206637998294
bummer, -2.4854141948688393
sucks, -2.565759323143935
sad, -3.1297322599724455
inaperfectworld, -3.4437137640059095


## Model 2. TFIDF & Logistic Regression

Ещё один способ работы с текстовыми данными — TF-IDF (Term Frequency–Inverse Document Frequency). Как и в bag-of-words, признаками будут являться слова, но теперь каждый текст будет описываться не векотром с количеством вхождений каждого из слов, а некоторй статистикой tfidf. Рассмотрим коллекцию текстов $D$. Для каждого уникального слова $t$ из документа $d \in D$ вычислим следующие величины:

1. Term Frequency – количество вхождений слова в отношении к общему числу слов в тексте:  

$$\text{tf}(t, d) = \frac{n_{td}}{\sum_{t \in d} n_{td}},$$

где $n_{td}$ — количество вхождений слова $t$ в текст $d$.
2. Inverse Document Frequency - инверсия частоты, с которой некоторое слово встречается в документах коллекции

$$\text{idf}(t, D) = \log \frac{\left| D \right|}{\left| \{d\in D: t \in d\} \right|},$$ 

где $\left| \{d\in D: t \in d\} \right|$ – количество текстов в коллекции, содержащих слово $t$, а $\left|D\right|$ - количество докуметов.


Тогда для каждой пары (слово, текст) $(t, d)$ вычислим величину: 

$$\text{tf-idf}(t,d, D) = \text{tf}(t, d)\cdot \text{idf}(t, D).$$

Отметим, что значение $\text{tf}(t, d)$ корректируется для часто встречающихся общеупотребимых слов при помощи значения $\text{idf}(t, D).$


In [50]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [52]:
vectorizer = TfidfVectorizer(min_df=5)
_ = vectorizer.fit(texts)

In [53]:
X = vectorizer.transform(texts)

In [54]:
cv = ShuffleSplit(X.shape[0], n_iter=3, test_size=0.3)
for train_ids, test_ids in cv:
    lr = LogisticRegression()
    lr.fit(X[train_ids], Y[train_ids])
    preds = lr.predict_proba(X[test_ids])[:,1]
    print('ROC-AUC: %.3f, ACC: %.3f' % (roc_auc_score(Y[test_ids], preds), 
                                        accuracy_score(Y[test_ids], (preds > 0.5).astype(int))))

ROC-AUC: 0.851, ACC: 0.773
ROC-AUC: 0.848, ACC: 0.769
ROC-AUC: 0.849, ACC: 0.771


In [69]:
f_weights = zip(vectorizer.get_feature_names(), lr.coef_[0])
f_weights = sorted(f_weights, key=lambda i: i[1])
for i in range(1,30):
    print("{0}, {1}".format(f_weights[-i][0], f_weights[-i][1]))
    
print('...')
for i in reversed(range(1,30)):
    print("{0}, {1}".format(f_weights[i][0], f_weights[i][1]))

thanks, 4.98617560872693
thank, 4.171159804580989
welcome, 4.001053433432517
followfriday, 3.9934022824762434
great, 3.8782399852062706
wait, 3.5130270247128754
musicmonday, 3.4573661739856973
awesome, 3.3101598804289356
worries, 3.2736362581684317
amazing, 3.0613245804017315
yay, 3.050170538706173
good, 3.02087464694146
sweet, 2.9639760673966355
hehe, 2.9310283810101008
happy, 2.9267614358149694
cool, 2.877626189806358
congratulations, 2.8146480394775564
love, 2.74640082883442
glad, 2.731931461400761
you, 2.730167384923653
haha, 2.7188719727051476
congrats, 2.7069063292305846
nice, 2.695781643228622
smile, 2.6646671867799316
hi, 2.628709592123032
worry, 2.6046885574117415
cute, 2.5649139419820677
enjoy, 2.5621717801501993
hello, 2.510816830623742
...
damn, -3.1957719301698924
raining, -3.196937268578495
broke, -3.2263360201451623
doesnt, -3.240518825092542
upset, -3.2697723364047846
horrible, -3.2719409020221
bummer, -3.38222258783503
no, -3.458224688760571
cry, -3.496476229089421
sig

## Улучшение модели 

Что можно попробовать? Каждый текст содержит слова, которые никак не влияют на смысл, в английском языке такими словами могут являться предлоги, союзы, местоимения  и тп. Попробуем удалить такие слова и посмотреть на результат классификации.

In [78]:
import nltk


In [137]:
nltk.download("stopwords")

In [72]:
from nltk.corpus import stopwords

In [73]:
stop_words = stopwords.words('english')

In [76]:
stop_words[:10]

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your']

#### Задание 1:
Удалите из текста stop-words, обучите и протестируйте модель на кросс-валидации

In [ ]:
# your code there

Удалось улучшить результат?

#### Ответ:

Что еще? Как в английском, так и в русском языках слова имеют различные формы, чтобы не учитывать каждую форму в модели, хотелось бы однокоренные слова привести к одному слову. Разберем два популярных способа: стемминг и лемматизация.

### Стемминг

Stemming – это процесс нахождения основы слова. В результате применения данной процедуры однокоренные слова, как правило, преобразуются к одинаковому виду.

In [79]:
stemmer = nltk.stem.snowball.EnglishStemmer()
print(stemmer.stem("cats"), stemmer.stem("smiling"))

cat smile


In [84]:
texts[1]

'                   i missed the new moon trailer'

In [85]:
sen = " ".join(list(map(lambda x: stemmer.stem(x), texts[1].split())))
print(sen)

i miss the new moon trailer


#### Задание 2:
Воспользуйтесь стеммингом для обработки слов в текстах, обучите и протестируйте модель на кросс-валидации.

In [135]:
# your code here

Удалось улучшить результат?

#### Ответ:

### Лемматизация

Лемматизация — процесс приведения слова к его нормальной форме (лемме)

In [138]:
nltk.download("wordnet")

In [139]:
nltk.download('punkt')
nltk.download('maxent_treebank_pos_tagger')

In [140]:
nltk.download('averaged_perceptron_tagger')

In [88]:
from nltk.stem.wordnet import WordNetLemmatizer

In [89]:
from nltk.corpus import wordnet as wn

In [90]:
lmtzr = WordNetLemmatizer()

In [91]:
def is_noun(tag):
    return tag in ['NN', 'NNS', 'NNP', 'NNPS']


def is_verb(tag):
    return tag in ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']


def is_adverb(tag):
    return tag in ['RB', 'RBR', 'RBS']


def is_adjective(tag):
    return tag in ['JJ', 'JJR', 'JJS']


def penn_to_wn(tag):
    if is_adjective(tag):
        return wn.ADJ
    elif is_noun(tag):
        return wn.NOUN
    elif is_adverb(tag):
        return wn.ADV
    elif is_verb(tag):
        return wn.VERB
    return None

In [133]:
texts[3]

'           omgaga im sooo  im gunna cry ive been at this dentist since  i was suposed  just get a crown put on mins'

In [161]:
pos = nltk.pos_tag(texts[3].split())
sen = " ".join([lmtzr.lemmatize(p[0], penn_to_wn(p[1])) for p in pos if penn_to_wn(p[1]) != None])
print(sen)

omgaga im sooo im gunna cry ive be dentist i be suposed just get crown put min


#### Задание 3: 
Воспользуйтесь лемматизацией для обработки слов в текстах, обучите и протестируйте модель на кросс-валидации.

In [141]:
# your code here

Удалось улучшить результат?

#### Ответ:

#### Задание 4:
Кратко опишите результаты. Какой метод оказался самым эффективным?

#### Ответ: